# TM10007 Assignment template

In [47]:
# Run this to use from colab environment
!git clone https://github.com/Doesjka/TM10007_ML_g9.git

fatal: destination path 'TM10007_ML_g9' already exists and is not an empty directory.


## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [48]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import decomposition
import seaborn
import matplotlib.pyplot as plt

data = pd.read_csv('/content/TM10007_ML_g9/worclipo/Lipo_radiomicFeatures.csv', index_col=0)
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')


The number of samples: 115
The number of columns: 494


## Splitting data in train and test set


In [49]:
y = data['label']
X = data.drop('label', axis=1)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)


## Handling missing data 
### Throwing out features
All features that exist of at least 50% zeros are deleted from the data. 


In [50]:
zeros = (X_train == 0).sum()
threshold = 0.5 * len(y_train)
print('Threshold = ', threshold)
feature_del = zeros[zeros > threshold]

X_train = X_train.drop(columns=feature_del.index)
print(f'{len(data.columns)-len(X_train.columns)} features were deleted')

Threshold =  43.0
25 features were deleted


In [51]:
more_zeros = (X_train == 0).sum()
columns_zeros = more_zeros[more_zeros > 0].index
print(f'Of the remaining features, {len(columns_zeros)} features have at least one zero')
print(f'There is a total of {more_zeros.sum()} zeros left in the data')


Of the remaining features, 10 features have at least one zero
There is a total of 44 zeros left in the data


## Calculate number of missing values per sample

In [52]:
zeros_r = (X_train == 0).sum(axis=1)
threshold = 0.005 * X_train.size / len(y_train)
print('Threshold = ', threshold)
feature_del = zeros_r[zeros_r > threshold]
print(feature_del)

Threshold =  2.345
ID
Lipo-090_0    5
Lipo-095_0    3
Lipo-076_0    3
Lipo-003_0    3
dtype: int64


In [53]:
from scipy.stats import shapiro

aantal_normaal = 0

for column in X_train.columns:
    result = shapiro(X_train[column])
    # print(result.pvalue)
    normaal = result.pvalue > 0.05
    aantal_normaal += normaal
    
print(aantal_normaal)



74


/usr/local/lib/python3.9/dist-packages/scipy/stats/_morestats.py:1813: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


### Filling remaining zeros
All remaining zeros are replaced by the mean of that feature. 

In [54]:
X_train_mean = X_train
X_train_median = X_train

for column in columns_zeros[:]:
    print('Kolom: ', column)
    column_mean = X_train.loc[X_train[column]!=0, column].mean()
    column_median = X_train.loc[X_train[column]!=0, column].median()
    print('mean = ', column_mean)
    print('median = ', column_median)

    verschil2_percolumn = column_mean - column_median
    print('Verschil tussen mean en median = ',verschil2_percolumn)

    # result = shapiro(X_train[column])
    # result.pvalue
    print('p-waarde normaalverdeling = ', shapiro(X_train.loc[X_train[column]!=0, column]).pvalue)

    X_train_mean[column].replace(0, column_mean)
    X_train_median[column].replace(0, column_median)

    print(' ')

verschil = abs(X_train_mean - X_train_median)
print('Totaal verschil = ', verschil.sum().sum())

# X_train.head()
# pd.set_option('display.max_rows', None)
# print(X_train[columns_zeros[0]])
# pd.set_option('display.max_rows', 10)


Kolom:  PREDICT_original_sf_area_min_2.5D
mean =  456.8698899846219
median =  200.39520811289555
Verschil tussen mean en median =  256.4746818717264
p-waarde normaalverdeling =  3.05788068477586e-11
 
Kolom:  PREDICT_original_tf_LBP_quartile_range_R8_P24
mean =  9.301204819277109
median =  11.0
Verschil tussen mean en median =  -1.6987951807228914
p-waarde normaalverdeling =  1.931198130478151e-05
 
Kolom:  PREDICT_original_vf_Frangi_full_quartile_range_SR(1.0, 10.0)_SS2.0
mean =  2.766293780856362e-07
median =  4.602205232532876e-10
Verschil tussen mean en median =  2.7616915756238295e-07
p-waarde normaalverdeling =  9.707482768662333e-20
 
Kolom:  PREDICT_original_vf_Frangi_edge_quartile_range_SR(1.0, 10.0)_SS2.0
mean =  2.766293780856362e-07
median =  4.602205232532876e-10
Verschil tussen mean en median =  2.7616915756238295e-07
p-waarde normaalverdeling =  9.707482768662333e-20
 
Kolom:  PREDICT_original_vf_Frangi_inner_quartile_range_SR(1.0, 10.0)_SS2.0
mean =  8.705055316112096e-

## Scaling

In [55]:
scaler = preprocessing.StandardScaler().fit(X_train_median)
X_train_scaled = scaler.transform(X_train_median)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_median.columns)

# scaler = preprocessing.RobustScaler().fit(X_train_median)
# X_train_scaled = scaler.transform(X_train_median)
# X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_median.columns)

# scaler = preprocessing.MinMaxScaler().fit(X_train_median)
# X_train_scaled = scaler.transform(X_train_median)
# X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_median.columns)

In [ ]:
import scipy.stats as stats
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 10)

# stats.zscore(X_train)    # Dit is voor een array 
Z_score = X_train.apply(stats.zscore)    # Dit is voor een pandas dataframe 

num_outliers = (np.abs(Z_score) > 3).sum()
outliers = num_outliers.sum()
numb_columns_outliers = (num_outliers > 0).sum()

print(num_outliers)
print(numb_columns_outliers)
print(outliers)

## PCA

In [56]:
pca = decomposition.PCA()
pca.fit(X_train_scaled_df)
X_pca = pca.transform(X_train_scaled_df)

component=0
total_ratio = 0
while total_ratio < 0.9:
    total_ratio += pca.explained_variance_ratio_[component]
    component+=1

X_pca = X_pca[:,0:component]

## k-NN

In [65]:
from sklearn import neighbors
# Specify the search range, this could be multiple parameters for more complex classifiers
parameters = {'n_neighbors': list(range(1, 26, 2)),
              'weights': ['uniform', 'distance'],
              'p': [1, 2]}

# Specify the cross validation method to use, we use 10-fold stratified cross-validation
cv_10fold = model_selection.StratifiedKFold(n_splits=10)

# Create the grid search method, use area under ROC curve as scoring metric
# Too learn more about metrics see: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
clf = model_selection.GridSearchCV(neighbors.KNeighborsClassifier(), parameters, cv=cv_10fold, scoring='accuracy')

# Do the entire search
clf.fit(X_pca, y_train)

# Show the complete results of the cross validation
clf_df = pd.DataFrame(clf.cv_results_)

# Extract the best k 
clf_df = clf_df.sort_values(by=['rank_test_score'])
optimal_k = clf_df['param_n_neighbors'].iloc[0]
display(clf_df)

# Extract the best hyperparameters 
clf.best_params_



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
25,0.000831,0.000022,0.001432,0.000124,13,1,distance,"{'n_neighbors': 13, 'p': 1, 'weights': 'distan...",0.777778,0.333333,...,0.666667,0.555556,0.777778,0.750,0.625,0.625,0.750,0.641667,0.130998,1
24,0.000878,0.000056,0.001717,0.000096,13,1,uniform,"{'n_neighbors': 13, 'p': 1, 'weights': 'uniform'}",0.777778,0.333333,...,0.666667,0.555556,0.777778,0.750,0.625,0.625,0.750,0.641667,0.130998,1
40,0.000902,0.000059,0.002328,0.001368,21,1,uniform,"{'n_neighbors': 21, 'p': 1, 'weights': 'uniform'}",0.555556,0.555556,...,0.333333,0.555556,0.888889,0.625,0.875,0.625,0.625,0.630556,0.152677,3
42,0.001128,0.000437,0.002507,0.001521,21,2,uniform,"{'n_neighbors': 21, 'p': 2, 'weights': 'uniform'}",0.555556,0.555556,...,0.555556,0.666667,0.888889,0.750,0.500,0.875,0.500,0.629167,0.150878,4
43,0.001003,0.000031,0.001767,0.000109,21,2,distance,"{'n_neighbors': 21, 'p': 2, 'weights': 'distan...",0.666667,0.555556,...,0.555556,0.555556,0.888889,0.750,0.500,0.875,0.500,0.629167,0.150878,4
29,0.000871,0.000088,0.001405,0.000029,15,1,distance,"{'n_neighbors': 15, 'p': 1, 'weights': 'distan...",0.555556,0.333333,...,0.555556,0.666667,0.777778,0.750,0.750,0.750,0.625,0.620833,0.141100,6
28,0.000989,0.000323,0.003001,0.002883,15,1,uniform,"{'n_neighbors': 15, 'p': 1, 'weights': 'uniform'}",0.555556,0.333333,...,0.555556,0.666667,0.777778,0.750,0.750,0.750,0.625,0.620833,0.141100,6
41,0.001016,0.000332,0.001964,0.001309,21,1,distance,"{'n_neighbors': 21, 'p': 1, 'weights': 'distan...",0.555556,0.444444,...,0.333333,0.555556,0.888889,0.625,0.875,0.625,0.625,0.619444,0.161518,8
39,0.000879,0.000044,0.001501,0.000052,19,2,distance,"{'n_neighbors': 19, 'p': 2, 'weights': 'distan...",0.666667,0.444444,...,0.333333,0.555556,0.777778,0.625,0.750,0.875,0.500,0.619444,0.155803,8
21,0.000908,0.000027,0.002002,0.001418,11,1,distance,"{'n_neighbors': 11, 'p': 1, 'weights': 'distan...",0.666667,0.333333,...,0.666667,0.555556,0.888889,0.750,0.375,0.625,0.750,0.616667,0.161518,10


{'n_neighbors': 13, 'p': 1, 'weights': 'uniform'}

## Random Forest

In [60]:
from scipy.stats import randint
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

# Create the random grid
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

parameters = {'n_estimators': randint(1, 400),
              'max_depth': max_depth,
              'min_samples_split': randint(2, 11),
              'min_samples_leaf': randint(1, 5)}
              
clf = model_selection.RandomizedSearchCV(RandomForestClassifier(), parameters, cv=10, n_iter=20, scoring='accuracy')
    
# Fit the classifier
clf.fit(X_pca, y_train)

# Show the complete results of the cross validation
clf_df = pd.DataFrame(clf.cv_results_)
display(clf_df)

# Extract the best hyperparameters 
clf.best_params_


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.617002,0.152372,0.035776,0.010274,50,2,5,353,"{'max_depth': 50, 'min_samples_leaf': 2, 'min_...",0.444444,...,0.666667,0.444444,0.777778,0.875,0.500,0.875,0.500,0.597222,0.174359,15
1,0.178295,0.012127,0.013586,0.005273,90,1,3,104,"{'max_depth': 90, 'min_samples_leaf': 1, 'min_...",0.444444,...,0.777778,0.444444,0.888889,0.625,0.500,0.750,0.625,0.583333,0.170217,18
2,0.503436,0.229954,0.035927,0.022285,70,4,4,87,"{'max_depth': 70, 'min_samples_leaf': 4, 'min_...",0.666667,...,0.444444,0.444444,0.777778,0.875,0.500,0.875,0.625,0.609722,0.178455,9
3,0.214761,0.051316,0.016745,0.006432,70,1,5,86,"{'max_depth': 70, 'min_samples_leaf': 1, 'min_...",0.666667,...,0.666667,0.444444,1.000000,0.875,0.250,0.875,0.500,0.627778,0.222552,5
4,0.125576,0.006111,0.008473,0.000772,110,2,7,79,"{'max_depth': 110, 'min_samples_leaf': 2, 'min...",0.444444,...,0.666667,0.555556,0.555556,0.750,0.625,0.625,0.750,0.597222,0.102439,15
5,0.485712,0.068294,0.029620,0.005501,70,3,2,300,"{'max_depth': 70, 'min_samples_leaf': 3, 'min_...",0.555556,...,0.777778,0.444444,0.666667,0.750,0.625,0.750,0.625,0.608333,0.124969,11
6,0.615092,0.135565,0.038254,0.008528,60,4,9,355,"{'max_depth': 60, 'min_samples_leaf': 4, 'min_...",0.555556,...,0.888889,0.444444,0.777778,0.875,0.875,0.875,0.625,0.669444,0.203916,2
7,0.492583,0.009487,0.031027,0.006360,10,2,5,323,"{'max_depth': 10, 'min_samples_leaf': 2, 'min_...",0.555556,...,0.777778,0.555556,0.555556,0.750,0.375,0.875,0.625,0.606944,0.145807,12
8,0.276452,0.043847,0.019511,0.003823,70,3,9,115,"{'max_depth': 70, 'min_samples_leaf': 3, 'min_...",0.333333,...,0.555556,0.555556,0.888889,0.750,0.875,0.875,0.750,0.658333,0.186256,3
9,0.536799,0.065554,0.028765,0.001279,50,1,6,311,"{'max_depth': 50, 'min_samples_leaf': 1, 'min_...",0.444444,...,0.666667,0.444444,0.777778,0.875,0.500,0.750,0.625,0.597222,0.155407,15


{'max_depth': 50,
 'min_samples_leaf': 4,
 'min_samples_split': 7,
 'n_estimators': 208}

## SVM

In [59]:
# import scipy 
# from sklearn.svm import SVC
# from sklearn import model_selection

# svm = SVC()
# parameters = {'C': [1, 10, 100, 1000], 'kernel': ['linear']}
# #{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}]
 
# cv_10fold = model_selection.StratifiedKFold(n_splits=10)

# grid_search = model_selection.GridSearchCV(svm, parameters, cv=cv_10fold, scoring='roc_auc')

# # Do the entire search
# grid_search.fit(X_train_scaled, y_train)

# # Show the complete results of the cross validation
# pd.DataFrame(grid_search.cv_results_)
